In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib as plt
import scipy as sci

In [3]:
data_frames = pd.read_excel("rawData.xlsx", sheet_name=["stitch_1","stitch_2","gases_1","gases_2","temperature_1","temperature_2","all_night"], header=0)

### Описание: прогон 36 сенсора (прошивка новая, исправленная) в термокамере под азотом	( 1 эксперимент)
включение сенсора, подача азота, прогрев, постановка на запись						
охлаждение термокамеры до +10 град, выдержка 30 мин	
с 3119 сек включение режима +40 град с шагом около 5 град, чтобы замедлить скорость нагрева									
с 7194 сек полное выключение термокамеры, сенсор остывает на записи под азотом								

In [4]:
data_frames['stitch_1']

,t,Um,Ur,Ud
0,108,0.70172,0.70287,0.77656
1,113,0.70175,0.70305,0.77649
2,118,0.70177,0.70299,0.77649
3,123,0.70173,0.70329,0.77649
4,128,0.70214,0.70309,0.77651
...,...,...,...,...
1784,8984,0.50134,0.42593,0.73348
1785,8989,0.50162,0.42612,0.73348
1786,8994,0.50158,0.42604,0.73347
1787,8999,0.50189,0.42621,0.73343


### Предобработка: добавление температурных меток

In [5]:
def add_temp(df, temp_to_times):
    for ind, row in df.iterrows():
        for temp_to_time in temp_to_times:
            if (row['t'] < temp_to_time[0]) or (temp_to_time[0] == -1):
                df.loc[ind, "temperature"] = temp_to_time[1]
                break
    df["temperature"] = df["temperature"].astype(int)

In [6]:
temp_to_times = [(3119,10), (7194, 40), (-1, 20)]
add_temp(data_frames['stitch_1'], temp_to_times)
add_temp(data_frames['stitch_2'], temp_to_times)

In [7]:
data_frames['stitch_1']

,t,Um,Ur,Ud,temperature
0,108,0.70172,0.70287,0.77656,10
1,113,0.70175,0.70305,0.77649,10
2,118,0.70177,0.70299,0.77649,10
3,123,0.70173,0.70329,0.77649,10
4,128,0.70214,0.70309,0.77651,10
...,...,...,...,...,...
1784,8984,0.50134,0.42593,0.73348,20
1785,8989,0.50162,0.42612,0.73348,20
1786,8994,0.50158,0.42604,0.73347,20
1787,8999,0.50189,0.42621,0.73343,20


### Описание: прогон 36 сенсора (прошивка новая, исправленная) в термокамере под азотом	( 2 эксперимент)
остывание записывалось всю ночь					

In [8]:
data_frames['stitch_2']

,t,Um,Ur,Ud,temperature
0,108,0.70172,0.70287,0.77656,10
1,113,0.70175,0.70305,0.77649,10
2,118,0.70177,0.70299,0.77649,10
3,123,0.70173,0.70329,0.77649,10
4,128,0.70214,0.70309,0.77651,10
...,...,...,...,...,...
14444,72284,0.70094,0.70127,0.77651,20
14445,72289,0.70106,0.70140,0.77649,20
14446,72294,0.70090,0.70133,0.77656,20
14447,72299,0.70130,0.70127,0.77654,20


### Вопросы: 
Нет ли более детальной температурной разметки, потому что из имеющегося описания точно не понятно на какой температуре в какой момент времени производились измерения: 
* у охлаждения не указан температурный шаг и стартовая температура
* у нагревания не указан временной шаг ( сказано что шаг 5 градусов, но не сказано в какой период)
* остывание - нет температуры до которой происходило остывание, предположительно комнатная температура (20 градусов).  
Обобщение: для того чтобы делать выводы о работе сенсора в различных температурных условиях, требуются более точные временно-температурные метки.

### Описание:  последовательная продувка газами по 25 минут				
азот 	с 1592 сек  
0,05%	с 3042 сек  
0,20%	с 4502 сек  
0,50%	с 6032 сек  
1%	с 7522 сек  
2%	с 8962 сек  
3%	с 10582 сек  
5%	с 12087 сек  

In [9]:
data_frames['gases_1']

,t,C,Um,Ur,Ud
0,1592,84.065575,0.70109,0.70087,0.77666
1,1597,-202.939117,0.70105,0.70094,0.77664
2,1602,-208.769150,0.70101,0.70099,0.77669
3,1607,-284.524109,0.70081,0.70080,0.77653
4,1612,45.755302,0.70090,0.70072,0.77648
...,...,...,...,...,...
2461,13897,NaN,0.65410,0.67793,0.77440
2462,13902,NaN,0.65423,0.67784,0.77438
2463,13907,NaN,0.65418,0.67774,0.77443
2464,13912,NaN,0.65417,0.67787,0.77437


Количество строк с неизвестным значение C

In [10]:
data_frames['gases_1'].isna().sum()

t        0
C     1454
Um       0
Ur       0
Ud       0
dtype: int64

In [11]:
def add_perc(df, perc_to_times):
    for ind, row in df.iterrows():
        for perc_to_time in perc_to_times:
            if (row['t'] < perc_to_time[0]) or (perc_to_time[0] == -1):
                df.loc[ind, "percentage"] = perc_to_time[1]
                break

### Описание: сенсор 36 тестировался с газами 6 февраля 2020 года					
азот	с 70 сек	(13:46-14:50)	
0,05%	с 3730 сек	(14:50-15:10)	
0,20%	с 5100 сек	(15:12-15:30)	
2%	с 6300сек	(15:32-15:52)	
3%	с 7475сек	(15:52-16:12)	
5%	с 8695сек	(16:12-16:32)	
азот	с 9950сек	(16:33-16:53)  

In [12]:
data_frames['gases_2']

,t,C,Um,Ur,Ud
0,65,5.984166,0.70175,0.70086,0.77665
1,70,-14.744394,0.70181,0.70086,0.77667
2,75,-224.305252,0.70211,0.70073,0.77665
3,80,-49.504810,0.70185,0.70082,0.77666
4,85,-143.622314,0.70192,0.70067,0.77662
...,...,...,...,...,...
2294,11535,-124.971397,0.69357,0.68850,0.77439
2295,11540,-223.308426,0.69363,0.68834,0.77440
2296,11545,-370.392334,0.69373,0.68813,0.77444
2297,11550,-305.286804,0.69384,0.68844,0.77446


### Предобработка: добавление меток процента газа 

In [13]:
perc_to_times_1 = [(3042, 0),( 4502,0.05),(6032,0.2),(7522,0.5),(8962, 1),(10582, 2), (12087, 3), (-1, 5)]
perc_to_times_2 = [(3730, 0),( 5100,0.05),(6300,0.2),(7475,2),(8695, 3),(9950, 5), (-1, 0)]
add_perc(data_frames['gases_1'], perc_to_times_1)
add_perc(data_frames['gases_2'], perc_to_times_2)

In [14]:
data_frames['gases_1']

,t,C,Um,Ur,Ud,percentage
0,1592,84.065575,0.70109,0.70087,0.77666,0.0
1,1597,-202.939117,0.70105,0.70094,0.77664,0.0
2,1602,-208.769150,0.70101,0.70099,0.77669,0.0
3,1607,-284.524109,0.70081,0.70080,0.77653,0.0
4,1612,45.755302,0.70090,0.70072,0.77648,0.0
...,...,...,...,...,...,...
2461,13897,NaN,0.65410,0.67793,0.77440,5.0
2462,13902,NaN,0.65423,0.67784,0.77438,5.0
2463,13907,NaN,0.65418,0.67774,0.77443,5.0
2464,13912,NaN,0.65417,0.67787,0.77437,5.0


In [20]:
C_perc = 0
MSE = 0
for ind, row in data_frames['gases_2'].iterrows():
    if row['C'] < 0:
        C_perc = 0
    else:
        C_perc = row['C']/(10**4)
    MSE += (C_perc - row['percentage'])**2
MSE /= data_frames['gases_2'].shape[0]
MSE

0.21134864743570322

### Вопросы: 
* В первом эксперименте у более чем половины объектов отсутствует значение ppm газа (С). С учётом того, что эти два эксперимента не идентичны, хотелось бы узнать, нет ли какой-либо чёткой формулы расчёта параметра С из параметров Um и Ur. Конечно значения можно восстановить численно, но если есть формула, то хотелось бы воспользоваться ей.
* При просмотре данных также заметен определённый лаг между временем, которое указано как начальное для следующей долевого этапа и временем, когда значение С примет соответствующее значение. Возникает вопрос: связан ли данный лаг с тем, что доля метана в смеси изменяется постепенно или же это лаг на стороне сенсора?

### Описание: сенсор 36 испытываплся на саморазогрев без газов ( 1 эксперимент)

In [21]:
data_frames['temperature_1']

,t,C,Um,Ur,Um/Ur,Ud,a(Ud),b(Ud),c(Ud),n(Ud),ce,c0,da
0,81,159.714905,0.74032,0.75733,0.977540,0.78457,2.13396,1.81325,0.74036,2.66800,2.08384,0,0
1,86,-149.926392,0.74022,0.75714,0.977653,0.78444,2.13317,1.81246,0.74018,2.66721,2.08323,0,0
2,91,286.596893,0.73984,0.75676,0.977642,0.78447,2.13157,1.81086,0.74007,2.66561,2.08175,0,0
3,96,286.521606,0.73985,0.75657,0.977900,0.78432,2.13076,1.81005,0.74007,2.66480,2.08092,0,0
4,101,374.332703,0.73951,0.75650,0.977541,0.78437,2.13044,1.80973,0.73996,2.66448,2.08072,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,5921,340.239410,0.70140,0.70210,0.999003,0.77615,1.90985,1.58914,0.70195,2.44389,1.89814,0,0
1169,5926,-200.111526,0.70149,0.70204,0.999217,0.77623,1.90964,1.58893,0.70134,2.44368,1.89855,0,0
1170,5931,-364.911957,0.70143,0.70179,0.999487,0.77610,1.90867,1.58796,0.70078,2.44271,1.89813,0,0
1171,5936,-304.560913,0.70141,0.70199,0.999174,0.77620,1.90943,1.58872,0.70099,2.44347,1.89868,0,0


### Описание: сенсор 36 испытывался на саморазогрев и зависимость от внешней температуры без газов 4 февраля ( 2 эксперимент)

In [10]:
data_frames['temperature_2']

,t,C,Um,Ur,Um/Ur,Ud,a(Ud),b(Ud),c(Ud),n(Ud),ce,c0,da
0,86,132.061676,0.72602,0.73682,0.985342,0.78190,2.04846,1.72775,0.72606,2.58250,2.01265,0,0
1,91,-345.992676,0.72598,0.73637,0.985890,0.78183,2.04661,1.72590,0.72555,2.58065,2.01131,0,0
2,96,299.691498,0.72588,0.73612,0.986089,0.78176,2.04561,1.72490,0.72618,2.57965,2.00967,0,0
3,101,-379.048615,0.72554,0.73602,0.985761,0.78176,2.04521,1.72450,0.72499,2.57925,2.01045,0,0
4,106,404.348511,0.72554,0.73583,0.986016,0.78175,2.04442,1.72371,0.72619,2.57846,2.00847,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3241,16291,NaN,0.70218,0.70257,0.999445,0.77673,1.91169,1.59098,NaN,2.44573,NaN,0,0
3242,16296,NaN,0.70198,0.70248,0.999288,0.77674,1.91136,1.59065,NaN,2.44540,NaN,0,0
3243,16301,NaN,0.70202,0.70243,0.999416,0.77672,1.91117,1.59046,NaN,2.44521,NaN,0,0
3244,16306,NaN,0.70211,0.70205,1.000085,0.77671,1.90969,1.58898,NaN,2.44373,NaN,0,0


### Предобработка: удаление нулевых столбцов

In [22]:
print(set(data_frames['temperature_1']['c0']), set(data_frames['temperature_1']['da']),
      set(data_frames['temperature_2']['c0']), set(data_frames['temperature_2']['da']))
data_frames['temperature_1'].drop(['c0','da'], axis='columns', inplace=True)
data_frames['temperature_2'].drop(['c0','da'], axis='columns', inplace=True)

{0} {0} {0} {0}


In [23]:
data_frames['temperature_2']

,t,C,Um,Ur,Um/Ur,Ud,a(Ud),b(Ud),c(Ud),n(Ud),ce
0,86,132.061676,0.72602,0.73682,0.985342,0.78190,2.04846,1.72775,0.72606,2.58250,2.01265
1,91,-345.992676,0.72598,0.73637,0.985890,0.78183,2.04661,1.72590,0.72555,2.58065,2.01131
2,96,299.691498,0.72588,0.73612,0.986089,0.78176,2.04561,1.72490,0.72618,2.57965,2.00967
3,101,-379.048615,0.72554,0.73602,0.985761,0.78176,2.04521,1.72450,0.72499,2.57925,2.01045
4,106,404.348511,0.72554,0.73583,0.986016,0.78175,2.04442,1.72371,0.72619,2.57846,2.00847
...,...,...,...,...,...,...,...,...,...,...,...
3241,16291,NaN,0.70218,0.70257,0.999445,0.77673,1.91169,1.59098,NaN,2.44573,NaN
3242,16296,NaN,0.70198,0.70248,0.999288,0.77674,1.91136,1.59065,NaN,2.44540,NaN
3243,16301,NaN,0.70202,0.70243,0.999416,0.77672,1.91117,1.59046,NaN,2.44521,NaN
3244,16306,NaN,0.70211,0.70205,1.000085,0.77671,1.90969,1.58898,NaN,2.44373,NaN


Количество строк с неизвестным значение C 

In [24]:
data_frames['temperature_2'].isna().sum()

t           0
C        1933
Um          0
Ur          0
Um/Ur       0
Ud          0
a(Ud)       0
b(Ud)       0
c(Ud)    1933
n(Ud)       0
ce       1933
dtype: int64

### Вопросы: 
* Опять более половины объектов с неопределённым значением параметра С во втором эксперименте.
* В описании эксперимента сказано, что датчик испытывался на саморазогрев и зависимость от внешней температуры, но никаких температурных значений нет.

### Описание: сенсор 36 простоял в открытом виде(не в коробке) на воздухе всю ночь									

a |	b |	n |	ca | cb | cc | cd |	da
--- | --- | --- | --- | --- | --- | --- | ---
0,348 |	0,0585 | 0,92 |	2,9844 | -10,892 |	16,117 | -7,4538 |	0,0004
0,348 |	0,0585 | 0,92 |	0,09779 | 1,1911 |	-0,6938 | 0,3154 |	0,0014

$$ N= \frac{Um - da}{c(Ur)} $$
$$ c(Ur)= ca + cb*(Ur) + cc*(Ur)^{2} + cd*(Ur)^{3} $$
$$ C\%= \begin{cases}
10000*(\frac{-ln\left ( 1 - \frac{(1-N)}{b} \right )}{a})^{\frac{1}{n}} & \text{ if } x\leq 1 \\ 
-10000*(\frac{-ln\left ( 1 - \frac{(1-N)}{b} \right )}{a})^{\frac{1}{n}} & \text{ if } x> 1 
\end{cases} $$

In [25]:
data_frames['all_night']

,t,C,Um,Ur,Ud,a(Ud),b(Ud),c(Ud),n(Ud),ce,c0,da,c(Ur),N,C%
0,129,111.169632,0.70567,0.70691,0.77733,0.348,0.0585,0.70450,0.92,0,0,0.0014,0.705635,0.999483,185.496828
1,134,9.098622,0.70583,0.70684,0.77735,0.348,0.0585,0.70445,0.92,0,0,0.0014,0.705585,0.999781,72.706892
2,139,185.414459,0.70587,0.70739,0.77751,0.348,0.0585,0.70483,0.92,0,0,0.0014,0.705981,0.999277,267.615796
3,144,107.156784,0.70613,0.70756,0.77753,0.348,0.0585,0.70495,0.92,0,0,0.0014,0.706103,0.999472,189.896091
4,149,76.868965,0.70634,0.70779,0.77755,0.348,0.0585,0.70510,0.92,0,0,0.0014,0.706269,0.999534,165.393159
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11321,56734,29.550182,0.66601,0.64881,0.76865,0.348,0.0585,0.66467,0.92,0,0,0.0014,0.666306,0.998955,400.313975
11322,56739,-78.904655,0.66624,0.64883,0.76865,0.348,0.0585,0.66469,0.92,0,0,0.0014,0.666318,0.999282,265.403063
11323,56744,-109.317490,0.66623,0.64874,0.76861,0.348,0.0585,0.66462,0.92,0,0,0.0014,0.666264,0.999349,238.388066
11324,56749,-99.944748,0.66627,0.64881,0.76863,0.348,0.0585,0.66467,0.92,0,0,0.0014,0.666306,0.999345,239.931294


### Предобработка: удаление нулевых столбцов

In [26]:
print(set(data_frames['all_night']['c0']), set(data_frames['all_night']['ce']))
data_frames['all_night'].drop(['c0','ce'], axis='columns', inplace=True)

{0} {0}


In [27]:
print(set(data_frames['all_night']['da']), set(data_frames['all_night']['n(Ud)']),
      set(data_frames['all_night']['b(Ud)']), set(data_frames['all_night']['a(Ud)']))

{0.0014} {0.92} {0.0585} {0.348}


### Вопросы:
* Какой смысл у параметра Ud и важен ли он вообще при измерении доли газа? В документе сказано, что он выражает амплитуду импульсов напряжения, в excel-е он указан как температурный сигнал, но неясно как этот сигнал конвертируется в температурные значения.
* Что значат параметры (a	b	n	ca	cb	cc	cd	da) и вообще хотелось бы некоторых разъяснений по поводу параметров c(Ur)	и C% (если С - это показатель ppm газа, то C% вообще не очень ясный, размерность у него точно не процентная)  и формул их вычислений.
* "сенсор 36 простоял в открытом виде(не в коробке) на воздухе всю ночь" - правильно ли стоит понимать это, как то, что за эталонное значение метана стоит принимать его содержание в атмосфере?

In [70]:
for df_name in data_frames.keys():
    data_frames[df_name].to_pickle(f"./data_{df_name}.pkl")